## 파이썬으로 웹 크롤러 만들기 1일차

## ch1. 첫 번째 웹 스크레이퍼

### BeautlifulSoup 실행해보기

In [2]:
# beautifulsoup 실행해보기

from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page1.html') 
    #읽고자 하는 페이지 html 얻기
bs = BeautifulSoup(html.read(), 'html.parser') 
    #첫 변수는 html 텍스트, 두변째 변수는 bs의 구문 분석기
    #대부분 html.parser를 사용할 것
print(bs.h1) #bs에 있는 html파일의 h1태그를 읽기

<h1>An Interesting Title</h1>


In [3]:
bs

<html>
<head>
<title>A Useful Page</title>
</head>
<body>
<h1>An Interesting Title</h1>
<div>
Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
</div>
</body>
</html>

### 스크래핑의 예외 처리

스크래핑 시 html구문이 제대로 작성되지 못한 경우 (태그가 안 닫혀 있는 경우 등)에
스크래퍼가 예외처리를 하지 못하여 멈추는 경우가 빈번     
-> 어떻게 처리할 것인가?


**발생할 수 있는 에러의 종류 예시**
1. HTTP 에러
- 페이지를 찾을 수 없는 경우, url 해석에서 에러가 생긴 경우
- 서버를 찾을 수 없는 경우    
ex) 404 Page Not Found, 500 Internal Server Error

2. URL 에러
- url에 오타가 있을 때 

In [5]:
#사이트 접속을 통해 html 텍스트 파일을 가지고 올 수 없는 경우
#두 가지 상황을 고려하여 예외를 처리해보기
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError


try:
    html = urlopen('http://www.pythonscraping.com/pages/page1.html') 

except HTTPError as e: #HTTP error 발생 경우
    print(e)

except URLError as e: #URL error 발생 경우
    print('The server could not be found!')
    
else:
    print('It Worked!')

It Worked!


In [6]:
#html파일을 가져오는 것은 성공했으나 내용의 태그에 문제가 있을 때
#예로 h1태그의 내용이 그 내용을 가지고 오려 할 때는 문제가 생길것

#에시
print(bs.nonExistentTag) #내용이 없어서 None을 반환

None


C:\ProgramData\Anaconda3\lib\site-packages\bs4\element.py:1441: UserWarning: .nonExistentTag is deprecated, use .find("nonExistent") instead. If you really were looking for a tag called nonExistentTag, use .find("nonExistentTag")
  name=tag_name


In [7]:
print(bs.nonExistentTag.someTag)

AttributeError: 'NoneType' object has no attribute 'someTag'

In [9]:
#None 객체에 대한 예외처리를 진행하기

try:
    badContent = bs.nonExistingTag.anotherTag
except AttributeError as e: #html 문서안의 태그를 잘못 사용한 경우에
    print('Tag was not found, Attribute Error')
else:
    if badContent == None: #html 문서안의 태그를 호출했는데 아무 내용도 없는 경우에
        print('Tag was not found')
    else:
        print(badContent)

Tag was not found, Attribute Error


In [10]:
#위의 예외상황들을 종합적으로 고려하여 Title을 추출하는 함수를 작성해보기
#먼저 페이지에 접속할 수 없는 경우에 대한 예외 처리를 진행해야하고
#다음으로 내용에 잘못 접근하는 경우에 대한 예외처리를 해야함

from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

def getTitle(url):
    try:
        html = urlopen(url)
    except HTTPError as e: #사이트를 접근할 수 없는 경우에 대한 예외 처리
        return None
    try:
        bs = BeautifulSoup(html.read(), 'html.parser')
        title = bs.body.h1
    except AttributeError as e: 
        #body.h1 태그를 찾을 수 없는 경우에 대한 예외처리
        #혹은 사이트를 접속할 수 없어 None 객체에 잘못된 메서드를 사용하여 발생하는
        #AttributeError에 대한 예외처리
        return None
    return title

In [11]:
title = getTitle('http://www.pythonscraping.com/pages/page1.html')
if title == None:
    print('Title could not be found')
else:
    print(title)

<h1>An Interesting Title</h1>


## ch2. 고급 HTML 분석
- CSS 속성 중 class나 id를 이용하여 원하는 데이터만 추출해보기

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/warandpeace.html')
bs = BeautifulSoup(html.read(), 'html.parser')

In [5]:
bs
    #span태그 class 속성안에 red, green 변수가 존재

<html>
<head>
<style>
.green{
	color:#55ff55;
}
.red{
	color:#ff5555;
}
#text{
	width:50%;
}
</style>
</head>
<body>
<h1>War and Peace</h1>
<h2>Chapter 1</h2>
<div id="text">
"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>"
<p></p>
It was in July, 1805, and the speaker was the well-known <span class="green">Anna
Pavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya
Fedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man
of high rank and importance, who was the firs

In [7]:
#초록색 글씨로 써져있는 내용들만 가져오기

nameList = bs.findAll('span', {'class':'green'})
    #첫 인자는 태그 이름, 두번째 인자는 속성
    #위 조건에 부합하는 태그 이름과 속성을 가진 모든 경우를 찾음

In [8]:
nameList

[<span class="green">Anna
 Pavlovna Scherer</span>,
 <span class="green">Empress Marya
 Fedorovna</span>,
 <span class="green">Prince Vasili Kuragin</span>,
 <span class="green">Anna Pavlovna</span>,
 <span class="green">St. Petersburg</span>,
 <span class="green">the prince</span>,
 <span class="green">Anna Pavlovna</span>,
 <span class="green">Anna Pavlovna</span>,
 <span class="green">the prince</span>,
 <span class="green">the prince</span>,
 <span class="green">the prince</span>,
 <span class="green">Prince Vasili</span>,
 <span class="green">Anna Pavlovna</span>,
 <span class="green">Anna Pavlovna</span>,
 <span class="green">the prince</span>,
 <span class="green">Wintzingerode</span>,
 <span class="green">King of Prussia</span>,
 <span class="green">le Vicomte de Mortemart</span>,
 <span class="green">Montmorencys</span>,
 <span class="green">Rohans</span>,
 <span class="green">Abbe Morio</span>,
 <span class="green">the Emperor</span>,
 <span class="green">the prince</span>,
 

In [9]:
#찾은 태그 안의 내용, contents만 따오기

for name in nameList:
    print(name.get_text()) #BeautifulSoup 객체가 제공하는 get_text라른 메서드를 사용하여
        #태그를 지우고 태그 안의 내용만 가져오기
        #get_text는 최종적인 단계에서 사용하는 것을 추천. 이 전에 추가 작업을 하려면 태그를 남겨서 하는 것이 필수

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


### 부모, 자식 (자손), 형제
- html문서의 태그 위치를 이용해 내용을 추출하기 위해 사용하는 개념과 방법

In [11]:
html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html.read(), 'html.parser')

In [12]:
bs

<html>
<head>
<style>
img{
	width:75px;
}
table{
	width:50%;
}
td{
	margin:10px;
	padding:10px;
}
.wrapper{
	width:800px;
}
.excitingNote{
	font-style:italic;
	font-weight:bold;
}
</style>
</head>
<body>
<div id="wrapper">
<img src="../img/gifts/logo.jpg" style="float:left;"/>
<h1>Totally Normal Gifts</h1>
<div id="content">Here is a collection of totally normal, totally reasonable gifts that your friends are sure to love! Our collection is
hand-curated by well-paid, free-range Tibetan monks.<p>
We haven't figured out how to make online shopping carts yet, but you can send us a check to:<br/>
123 Main St.<br/>
Abuja, Nigeria
We will then send your totally amazing gift, pronto! Please include an extra $5.00 for gift wrapping.</p></div>
<table id="giftList">
<tr><th>
Item Title
</th><th>
Description
</th><th>
Cost
</th><th>
Image
</th></tr>
<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) frien

In [14]:
bs.find('table', {'id' : 'giftList'}).children

In [16]:
#자식들만 추출하기
for child in bs.find('table', {'id' : 'giftList'}).children:
    print(child)
    #여기서는 table 태그 id 속성의 giftList 변수 아래의 자식들만 선택
    #자식은 어떤 태그 바로 아래에 위치한 태그를 의미, 즉 한 단계 아래 태그만 해당
    #자손은 어떤 태그 아래에 있는 모든 태그들을 의미, 즉 한 단계 이상의 모든 태그들이 해당
    #교재 p.45의 tr, tr.gift#gift1, .... 만 table#giftList에 해당되는 것



<tr><th>
Item Title
</th><th>
Description
</th><th>
Cost
</th><th>
Image
</th></tr>


<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>


In [17]:
# 형제 다루기 next_siblings

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html.read(), 'html.parser')

for sibling in bs.find('table', {'id' : 'giftList'}).tr.next_siblings:
    print(sibling)
    #형제 태그는 동일한 위치에 속한 태그들을 의미
    #next_siblings는 현재 선택된 태그를 제외하고 다음의 형제 태그들 안의 내용들을 모두 출력
    #따라서 표의 제목에 해당하는 bs.table#giftList.tr을 제외하고 나머지 형제 태그들을 출력



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr

In [22]:
# 부모 다루기
html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html.read(), 'html.parser')

print(bs.find('img', {'src': '../img/gifts/img1.jpg'}).parent.previous_sibling.get_text())
    # 작동방식은 다음과 같다
    # bs.find('img', {'src': '../img/gifts/img1.jpg'}) 에 해당하는 태그와 내용을 찾는다
    # bs.find('img', {'src': '../img/gifts/img1.jpg'}).parent, 위 코드에서 찾은 태그의 부모 태그를 찾아서 태그와 내용을 반환한다
    # bs.find('img', {'src': '../img/gifts/img1.jpg'}).parent.previous_sibling, 위 코드에서 찾은 부모 태그와 동등한 위치에 있는 이전 형제 태그와 내용을 반환한다
    # bs.find('img', {'src': '../img/gifts/img1.jpg'}).parent.previous_sibling.get_text(), 태그 안에 해당하는 문자열 콘텐츠만 출력한다


$15.00



### 정규표현식과 BeautifulSoup
- 위치로 태그와 내용을 파악하기 어려우나 어떤 규칙에 의거해서 내용을 추출하고자 하는 경우 자주 사용

In [33]:
#주어진 html에서 이미지 파일만 추출하고자 하는 경우
import re

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html.read(), 'html.parser')

images = bs.findAll('img', {'src': re.compile(r"\.\.\/img\/gifts\/img.*\.jpg")})
    #img 태그 안의 src 속성의 정규표현식으로 표현되는 모든 변수들의 태그와 내용을 찾은 images 객체
    
for image in images:
    print(image['src']) 
    #images 객체 안에 있는 태그들의 모음들을 하나씩 불러와서
    #그 안의 src 속성을 불러와 속성 안의 변수들을 읽기
    #이런 경우와 같이 태그 안의 contents와 달리 속성안에 우리가 찾고자 하는 어떤 값이 들어 있는 경우도 많음

<img src="../img/gifts/img1.jpg"/>
<img src="../img/gifts/img2.jpg"/>
<img src="../img/gifts/img3.jpg"/>
<img src="../img/gifts/img4.jpg"/>
<img src="../img/gifts/img6.jpg"/>


### 람다 표현식을 BeautifulSoup에 사용하기

In [41]:
# 태그에 있는 속성의 개수를 이용하여 특정한 태그를 찾고자 하는 경우
# findAll에 사용가능

bs.findAll(lambda tag: len(tag.attrs) == 2)
    #lambda의 결과가 True, False로 나타나는 경우에만 람다식을 이용하여 인자로 사용할 수 있음

[<img src="../img/gifts/logo.jpg" style="float:left;"/>,
 <tr class="gift" id="gift1"><td>
 Vegetable Basket
 </td><td>
 This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
 <span class="excitingNote">Now with super-colorful bell peppers!</span>
 </td><td>
 $15.00
 </td><td>
 <img src="../img/gifts/img1.jpg"/>
 </td></tr>,
 <tr class="gift" id="gift2"><td>
 Russian Nesting Dolls
 </td><td>
 Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
 </td><td>
 $10,000.52
 </td><td>
 <img src="../img/gifts/img2.jpg"/>
 </td></tr>,
 <tr class="gift" id="gift3"><td>
 Fish Painting
 </td><td>
 If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
 </td><td>
 $10,005.00
 </td><td>
 <img src="../img/gifts/img3.jpg"/>
 </td>

In [42]:
bs.findAll(lambda tag: tag.get_text() == 'Or maybe he\'s only resting?')
    #태그 안의 contents가 Or maybe he\'s only resting?인 태그만 추출하도록 lambda식을 사용
    #이를 응용하면 람다식과 정규표현식을 동시에 이용하여 정규표현식에 대응되는 태그만 찾을 수 있음!

[<span class="excitingNote">Or maybe he's only resting?</span>]